In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor


"""
__author__ = 'AndreasBrotzer'

##_____________________________________________________________
'''---- import libraries ----'''

import obspy as obs
import sys
import matplotlib.pyplot as plt

from numpy import floor


##_____________________________________________________________
''' ---- set variables ---- '''
 

## ask user to specify event information
name        = input("\nEnter a name:  ")
dt          = input("\nEnter date and time (e.g. 2020-06-29 09:52):  ")
timeframe   = input("\nEnter duration (in minutes):  ")  # in minutes
freq      = input("\nEnter corner frequencies (in Hz):  ") # in Hz


## check if input is provided
if not name or not dt or not timeframe:
    print('\n!! At least one parameter is missing!!')
    sys.exit()   
else:

    timeframe = float(timeframe)    

    timestamp = obs.UTCDateTime(dt)

    outname = name+'_'+str(timestamp.year)+str(timestamp.month)+str(timestamp.day)

    day = timestamp.julday

    year = timestamp.year
    
    freqs  = [float(f) for f in freq.split(',')]

    flabel = [str(f) for f in freq.split(',')]
    
    
##_____________________________________________________________
''' ---- settings ---- '''

## setting sample frequency B = 20Hz or H = 200 Hz
sr = 'B'  ## B or H


## data archive directory
path="/import/freenas-ffb-01-data/romy_archive/{}/BW/ROMY/".format(str(year))



##_____________________________________________________________
'''---- define methods ----'''

## read data 
def get_stream_data_event(path,comp,day,year,samplerate,beg,end):
    ''' get stream from romy archive daily files for one component and one event'''

    if comp == 'Z':
        name = '{}J{}.D/BW.ROMY.10.{}J{}.D.{}.'.format(str(samplerate),comp,str(samplerate),comp,year)
        st = obs.read(path+name+str(day),starttime=beg,endtime=end)

    else:
        name = '{}J{}.D/BW.ROMY..{}J{}.D.{}.'.format(str(samplerate),comp,str(samplerate),comp,year)
        st = obs.read(path+name+str(day),starttime=beg,endtime=end)
        
    return st
  

## save several plots into a pdf file with specific name
def save_to_pdf(figs,name):
    ''' Saving figures in array figs continously to pdf-file''' 
    
    import matplotlib.backends.backend_pdf
    pdf = matplotlib.backends.backend_pdf.PdfPages("/home/brotzer/spyder/figs/romy_{}.pdf".format(name))
    
    for f in figs: 
        pdf.savefig( f )
    pdf.close()


##_____________________________________________________________

def __main__():

    
    ## _______________________________________________________________
    ## get data of ring components for event 
    zste = get_stream_data_event(path,'Z',day,year,sr,timestamp,timestamp+timeframe*60)
    uste = get_stream_data_event(path,'U',day,year,sr,timestamp,timestamp+timeframe*60)
    vste = get_stream_data_event(path,'V',day,year,sr,timestamp,timestamp+timeframe*60)
    wste = get_stream_data_event(path,'W',day,year,sr,timestamp,timestamp+timeframe*60)

    ## combine streams to plot traces together
    ste = zste + uste + vste + wste
    
    #plt.style.use('default')
    
    fig0 = ste.plot(numer_of_ticks=6)

    

    ## _______________________________________________________________
    ## dayplot for specific event 
#    fig1=zste.plot(type='dayplot',events={'min_magnitude': floor(mag)});
#    fig2=uste.plot(type='dayplot',events={'min_magnitude': floor(mag)})
#    fig3=vste.plot(type='dayplot',events={'min_magnitude': floor(mag)})
#    fig4=wste.plot(type='dayplot',events={'min_magnitude': floor(mag)})
 


    ## _______________________________________________________________
    ## second plot of traces with filter
    
    
    
    fig1, axes = plt.subplots(4,1,figsize=(15,10), sharex='col')
    
    
    plt.style.use('seaborn')

    plt.subplots_adjust(hspace=0.2,wspace=0.2)
    
    ## _______________________________________________
        

    data = [zste, uste, vste, wste]
    channel = [zste[0].stats.channel, uste[0].stats.channel, vste[0].stats.channel, wste[0].stats.channel]

    
    from andbro__filter_traces import __filter_traces
    
    
    for i,comp in enumerate(data):
        
 
    
        try:
            comp = __filter_traces(comp, freqs)    
        except:
            freq = [0.01,0.1]
            comp = __filter_traces(comp, freq)    
            
    
        axes[i].plot(comp[0].times()/60,comp[0].data / abs(comp[0].data).max(),'k' , label=channel[i])
    
    
        axes[i].set_ylabel('rotaion rate (rads/s)')    
        #axes[i].set_ylim(-1,1)
        axes[i].set_xlim(0,timeframe)

        axes[i].legend()

        if i == len(data)-1:
            axes[i].set_xlabel('Time from {} UTC (min)'.format(dt))
            axes[0].set_title('filterd: {} - {} Hz '.format(flabel[0],flabel[1]))
    plt.show();
    

    ## _______________________________________________________________
    ## check if figure should be saved
    from andbro__save_to import __save_to

    save = input('\nSAVE FIGURE (y/n) ??:  ')
    
    if save == 'y':
        outpath = input('\nEnter path: ')
        outname = input('\nEnter name: ')
        form    = input('\nEnter format: ')
        
        #__save_to([fig0,fig1], str(outpath), str(outname), str(form))
        __save_to([fig1], str(outpath), str(outname), str(form))

    else: 
        print('\nFigure discared!\n')


__main__()

##_____________________________________________________________
# End of File

